In [ ]:
# Importing the required libraries, custom libraries and setting the python path
from load_env_and_set_pythonpath import print_pythonpath
import os
import inversion_utils as iu
import helita_io_lp as lp
import time
import numpy as np
from skimage.transform import resize
import ambig_utils as au
from sunpy.coordinates import sun
from astropy.coordinates import SkyCoord
import astropy.units as u
from sunpy.coordinates import frames
import numpy as np
print_pythonpath()

In [ ]:
# Path of the ambig executable obtained from the compiling the ME0-2.1.1 Fortran code
ambig_executable_path = '/mn/stornext/u3/avijeetp/codes/ambiguity_resolution/ME0-2.1.1/'
# Path of the ambig_par file which is the input file for the ambig executable
ambig_par = '/mn/stornext/u3/avijeetp/codes/pyMilne/ITA/ambig_par'
# Load the fcube files obtained from the inversion
# data_dir = '/mn/stornext/d20/RoCS/aditi/project/sst/inversion/pyMilne/ITA/temp3/'
# fblos = 'Blos_6173_2021-06-22_T081750_2021-06-22_T090813_t_0-162.fcube'
# fbhor = 'Bhor_6173_2021-06-22_T081750_2021-06-22_T090813_t_0-162.fcube'
# fbazi = 'Bazi_6173_2021-06-22_T081750_2021-06-22_T090813_t_0-162.fcube'
# fblos = 'Blos_6173_2020-08-07_T083019_2020-08-07_T083019_t_12-12.fcube'
# fbhor = 'Bhor_6173_2020-08-07_T083019_2020-08-07_T083019_t_12-12.fcube'
# fbazi = 'Bazi_6173_2020-08-07_T083019_2020-08-07_T083019_t_12-12.fcube'

# data_dir = '/mn/stornext/d9/data/avijeetp/1_Projects/33_SST/2022-06-26/'

fbhor = 'Bhor_6173_2022-06-26_T073626_2022-06-26_T094322_t_0-189.fcube'
fblos = 'Blos_6173_2022-06-26_T073626_2022-06-26_T094322_t_0-189.fcube'
fbazi = 'Bazi_6173_2022-06-26_T073626_2022-06-26_T094322_t_0-189.fcube'

In [ ]:
# Load the configuration from the JSON file
input_config = iu.load_yaml_config('input_config.yaml')
# Check the input configuration
config = iu.check_input_config(input_config, pprint=False, confirm=False)

In [ ]:
# Load the configuration from the JSON file
crop = config['crop']
xorg = config['xorg']
xsize = config['xsize'] 
yorg = config['yorg']
ysize = config['ysize']
rescale = config['rescale']
is_north_up = config['is_north_up']
fov_angle = config['fov_angle']
fits_info = config['fits_info']
verbose = config['verbose']
save_dir = config['save_dir']
pix = config['scale']

In [ ]:
# Create a unique prefix for the output files based on the current time
id = int(time.time())
prefix = '_'.join(fblos.split('_')[1:-2]) + '_' + str(id) + '_'
print(f'suffix: {prefix}')

In [ ]:
# Obtain the time index from the file name
time_index = int(fblos.split('_')[-1].split('-')[0])
print(f'time index: {time_index}')

In [ ]:
time_index = config['time_range'][0]
print(f'time index: {time_index}')

In [ ]:
# Get the range of latitudes and longitudes from the fits file info
tt = time_index
x1 = fits_info['hplnt'][tt][0]
x2 = fits_info['hplnt'][tt][1]
y1 = fits_info['hpltt'][tt][0]
y2 = fits_info['hpltt'][tt][1]
xcen = (x1 + x2) / 2
ycen = (y1 + y2) / 2
tobs = fits_info['all_start_times'][tt]

In [ ]:
# Load the data from the saved files
# data_dir = '/mn/stornext/d20/RoCS/aditi/project/sst/inversion/pyMilne/ITA/temp3/'
blos = lp.getdata(save_dir + fblos) # nx, ny, nt
bhor = lp.getdata(save_dir + fbhor)
bazi = lp.getdata(save_dir + fbazi)


In [ ]:
blos.shape

In [ ]:

# drop the redundant time axis
# blos = blos.squeeze().T # ny, nx
# bhor = bhor.squeeze().T
# bazi = bazi.squeeze().T
blos1 = blos[:,:,time_index].T
bhor1 = bhor[:,:,time_index].T
bazi1 = bazi[:,:,time_index].T
# Plot the data
iu.plot_image(blos1, figsize=(6, 6), title='Blos')

In [ ]:
np.min(bazi1), np.max(bazi1)    

In [ ]:
# Crop the data if needed
if crop:
    blos1 = blos1[yorg: yorg + ysize, xorg: xorg + xsize]
    bhor1 = bhor1[yorg: yorg + ysize, xorg: xorg + xsize]
    bazi1 = bazi1[yorg: yorg + ysize, xorg: xorg + xsize]
    if verbose:
        iu.plot_image(blos1, figsize=(6, 6), title='Blos after cropping')

In [ ]:
if rescale:
    ysize, xsize = blos1.shape 
    ysize = ysize // rescale
    xsize = xsize // rescale
    if ysize % 2 != 0:
        ysize += 1
    if xsize % 2 != 0:
        xsize += 1
    if verbose:
        print(f'Shape before rescale: {blos1.shape}')
    # Resample the array to the new size
    blos1 = resize(blos1, (ysize, xsize), anti_aliasing=True)
    bhor1 = resize(bhor1, (ysize, xsize), anti_aliasing=True)
    bazi1 = resize(bazi1, (ysize, xsize), anti_aliasing=True)
    if verbose:
        print(f'Shape after rescale: {blos1.shape}')
        iu.plot_image(blos1, figsize=(6, 6), title='Blos after rescaling')


In [ ]:

# Rotate the data to make it north-up
if not is_north_up:
    blos1 = iu.make_north_up(blos1, fov_angle)
    bhor1 = iu.make_north_up(bhor1, fov_angle)
    bazi1 = iu.make_north_up(bazi1, fov_angle)
    ysize, xsize = blos.shape 
    if verbose:
        print(f'Shape after rotation: {blos.shape}')
    if ysize % 2 != 0 or xsize % 2 != 0:
        print('Resampling the array to ensure that the shape is a multiple of 2')
        # Resample the array to ensure that the shape is a multiple of 2
        ysize = ysize // 2 * 2
        xsize = xsize // 2 * 2
        blos1 = resize(blos1, (ysize, xsize), anti_aliasing=True)
        bhor1 = resize(bhor1, (ysize, xsize), anti_aliasing=True)
        bazi1 = resize(bazi1, (ysize, xsize), anti_aliasing=True)
        if verbose:
            iu.plot_image(blos1, figsize=(6, 6), title='Blos after rotation')

In [ ]:
# Calculate the P, B0 and angular radius of the Sun, and convert the coordinates of center to Heliographic Stonyhurst longitude and latitude
pbr = sun.P(tobs).value, sun.B0(tobs).value, sun.angular_radius(tobs).value
coord = SkyCoord(xcen*u.arcsec, ycen*u.arcsec, obstime=tobs,
                 frame=frames.Helioprojective, observer='earth')
coord_hs = coord.transform_to(frames.HeliographicStonyhurst)
lonlat = coord_hs.lon.value, coord_hs.lat.value
print(f'P, B0, angular radius: {pbr}')
print(f'lon, lat: {lonlat}')

In [ ]:
# save the variables pix, pbr, lonlat, blos1, bhor1, bazi1 to a file 

np.savez('temp_ambig/ambig_input.npz', pix=pix, pbr=pbr, lonlat=lonlat, blos=blos1, bhor=bhor1, bazi=bazi1)

In [ ]:
blos1.shape

In [ ]:
# copy the ambig_par file to the save_dir with suffix appended
ambig_par_copy = save_dir  + prefix + 'par'
os.system(f'cp {ambig_par} {ambig_par_copy}')
if verbose:
    print(f'Copied {ambig_par} to {ambig_par_copy}')

In [ ]:
au.write_ambig_input('.', pix, pbr, lonlat, blos1, bhor1, bazi1)
# copy the ambig_input file to the save_dir with suffix appended
ambig_input_copy = save_dir + prefix + 'ambig_input.dat'
os.system(f'cp ./ambig_input.dat {ambig_input_copy}')
if verbose:
    print(f'Copied ./ambig_input.dat to {ambig_input_copy}')

In [ ]:
# Run the ambig executable with the ambig_par file as input
command = ambig_executable_path + 'ambig ' + ambig_par
au.run_command(command)
# Read the output files from the ambig executable
azimuth_dat = 'azimuth.dat'
# copy the azimuth.dat file to the save_dir with prefix appended to save_dir
azimuth_dat_copy = save_dir + prefix + 'azimuth.dat'
os.system(f'cp {azimuth_dat} {azimuth_dat_copy}')
if verbose:
    print(f'azimuth.dat copied to {azimuth_dat_copy}')
phi = au.read_azimuth_dat_file(azimuth_dat, (ysize, xsize))
# set the phi range to 0 to 2 pi instead of -pi to pi
phi = (phi + 2 * np.pi) % (2 * np.pi)
if verbose:
    iu.plot_image(phi, cmap='twilight', figsize=(6, 6), title='Azimuth angle')
    print(f'phi shape: {phi.shape}')
    print(f'phi min: {phi.min()}')
    print(f'phi max: {phi.max()}')

In [ ]:
# Calculate the magnetic field components from the azimuthal angle
bx = bhor1 * np.cos(phi)
by = bhor1 * np.sin(phi)
bz = blos1

In [ ]:
iu.plot_images([bx, by, bz, phi], title=['Bx', 'By','Bz', 'Phi'], fontsize=10, figsize=(20, 20), cmap=['gray', 'gray', 'gray', 'twilight'], grid_shape=(2, 2), fig_title=f'ID: {id}', save_fig=True, figname=save_dir + prefix + 'Bx_By_Bz_Phi.png')

---

In [ ]:
au.write_ambig_input('.', pix, pbr, lonlat, blos1, bhor1, phi)
# copy the ambig_input file to the save_dir with suffix appended
ambig_input_copy = save_dir + prefix + 'ambig_input.dat'
os.system(f'cp ./ambig_input.dat {ambig_input_copy}')
# Run the ambig executable with the ambig_par file as input
command = ambig_executable_path + 'ambig ' + ambig_par
au.run_command(command)
# Read the output files from the ambig executable
azimuth_dat = 'azimuth.dat'
# copy the azimuth.dat file to the save_dir with prefix appended to save_dir
azimuth_dat_copy = save_dir + prefix + 'azimuth.dat'
os.system(f'cp {azimuth_dat} {azimuth_dat_copy}')
if verbose:
    print(f'azimuth.dat copied to {azimuth_dat_copy}')
phi = au.read_azimuth_dat_file(azimuth_dat, (ysize, xsize))
# set the phi range to 0 to 2 pi instead of -pi to pi
phi2 = (phi + 2 * np.pi) % (2 * np.pi)
if verbose:
    iu.plot_image(phi, cmap='twilight', figsize=(6, 6), title='Azimuth angle')
    print(f'phi shape: {phi2.shape}')
    print(f'phi min: {phi2.min()}')
    print(f'phi max: {phi2.max()}')

In [ ]:
# Calculate the magnetic field components from the azimuthal angle
bx = bhor1 * np.cos(phi2)
by = bhor1 * np.sin(phi2)
bz = blos1
iu.plot_images([bx, by, bz, phi2], title=['Bx', 'By','Bz', 'Phi'], fontsize=10, figsize=(20, 20), cmap=['gray', 'gray', 'gray', 'twilight'], grid_shape=(2, 2), fig_title=f'ID: {id}', save_fig=True, figname=save_dir + prefix + 'Bx_By_Bz_Phi.png')

In [ ]:
# save bx, by, bz, phi as numpy files
np.save(save_dir + prefix + 'bx.npy', bx)
np.save(save_dir + prefix + 'by.npy', by)
np.save(save_dir + prefix + 'bz.npy', bz)
np.save(save_dir + prefix + 'phi.npy', phi)

---

In [ ]:
save_dir + prefix + 'bx.npy'


In [ ]:
# prefix = '6173_2020-08-07_T083019_2020-08-07_T083019_1721135870_'
# # restore the bx, by, bz from the numpy files
# bx = np.load(save_dir + prefix + 'bx.npy')
# by = np.load(save_dir + prefix + 'by.npy')
# bz = np.load(save_dir + prefix + 'bz.npy')

In [ ]:
from scipy.io import readsav

In [ ]:
savefile = '/mn/stornext/d9/data/avijeetp/1_Projects/61_QSEB_Topology/2020-08-07/AR12770/ambig/07_08_2020_h0822/AR12770_07_08_2020_h0822_nb_6173_ambig_1720793775_320_320_disambig_SFQ_Bxyz.sav'

In [ ]:
sav = readsav(savefile)
sav.keys()

In [ ]:
bx = sav['bx']
by = sav['by']
bz = sav['bz']

In [ ]:
iu.plot_images([bx, by, bz], title=['Bx', 'By','Bz'], fontsize=10, figsize=(12,5), cmap=['gray', 'gray', 'gray'], grid_shape=(1, 3), fig_title=f'ID: {id}', save_fig=True, figname=save_dir + prefix + 'Bx_By_Bz.png')

In [ ]:
# save bx, by, bz as numpy files
np.save(save_dir + prefix + 'sfq_bx.npy', bx)
print(f'saved {save_dir + prefix + "sfq_bx.npy"}')
np.save(save_dir + prefix + 'sfq_by.npy', by)
print(f'saved {save_dir + prefix + "sfq_by.npy"}')
np.save(save_dir + prefix + 'sfq_bz.npy', bz)
print(f'saved {save_dir + prefix + "sfq_bz.npy"}')